In [1]:
# from glob import glob
# import datetime as dt
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')

In [2]:
import xarray as xr
import json
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import useful_functions as uf

In [141]:
params = json.load(open(
    os.path.join('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow',
                 'outputs/dbpm_size_params.json')))

In [4]:
dbpm_input = pd.read_parquet(
    os.path.join('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/monthly_weighted',
                 'dbpm_clim-fish-inputs_fao-58_1841-2010.parquet'))

## Parameters from `sizemodel` function

In [29]:
ERSEM_det_input = False
temp_effect = True
use_init = False

In [5]:
# Input parameters to vary:
# time series of intercept of plankton size spectrum (estimated from GCM, 
# biogeophysical model output or satellite data).		
ui0 = 10**np.array(params['int_phy_zoo'])
[numb_size_bins] = params['numb_size_bins']
#Size bin index
size_bin_index = np.arange(0, numb_size_bins)
#Size bins
log10_size_bins = np.array(params['log10_size_bins'])
size_bin_vals = 10**log10_size_bins
[log10_pred_prey_ratio] = params['log10_pred_prey_ratio']
[log_prey_pref] = params['log_prey_pref']
[metabolic_req_pred] = params['metabolic_req_pred']
#Index for minimum detritivore size
[ind_min_detritivore_size] = params['ind_min_detritivore_size']
#Index for minimum predator size
[ind_min_pred_size] = params['ind_min_pred_size']
#Index for minimum size of detritivore fished
[ind_min_fish_det] = params['ind_min_fish_det']
#Index for minimum size of predator fished
[ind_min_fish_pred] = params['ind_min_fish_pred']
#Time
time = pd.date_range(start = min(dbpm_input.time)-DateOffset(months = 1),
                     end = max(dbpm_input.time), freq = 'MS')

# Initialising variables to store DBPM outputs

In [78]:
#data arrays for recording the two size spectra (V - det, U - pred)
predators = uf.init_da(log10_size_bins, time)
detritivores = uf.init_da(log10_size_bins, time)

#vector to hold detritus biomass density (g.m-2)
detritus = np.zeros(len(time))

#data arrays for keeping track of growth (GG_v, GG_u) and reproduction (R_v, R_u) 
#from ingested food:
reprod_det = uf.init_da(log10_size_bins, time)
reprod_pred = uf.init_da(log10_size_bins, time)
growth_det = uf.init_da(log10_size_bins, time)
growth_int_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of predation mortality (PM_v, PM_u)
pred_mort_det = uf.init_da(log10_size_bins, time)
pred_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of catches (Y_v, Y_u)
catch_det = uf.init_da(log10_size_bins, time)
catch_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of total mortality (Z_v, Z_u)
tot_mort_det = uf.init_da(log10_size_bins, time)
tot_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays to hold fishing mortality rates at each size class at time 
#(Fvec_v, Fvec_u)
fishing_mort_det = uf.init_da(log10_size_bins, time)
fishing_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of senescence mortality (SM_v, SM_u) and other 
#(intrinsic) mortality (OM_v, OM_u)
senes_mort_det = np.zeros(len(log10_size_bins))
senes_mort_pred = np.zeros(len(log10_size_bins))
other_mort_det = np.zeros(len(log10_size_bins))
other_mort_pred = np.zeros(len(log10_size_bins))

# Building lookup tables

In [7]:
#lookup tables for terms in the integrals which remain constant over time (gphi, mphi)
constant_growth = uf.gphi_f(uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                            log_prey_pref)
constant_mortality = uf.mphi_f(-uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                               log_prey_pref, metabolic_req_pred)

#lookup table for components of 10^(metabolic_req_pred*log10_size_bins) (expax)
met_req_log10_size_bins = uf.expax_f(log10_size_bins, metabolic_req_pred)

# Numerical integration

In [89]:
# set up with the initial values from param - same for all grid cells
#(phyto+zoo)plankton size spectrum (U) 
predators.isel(size_class = slice(None, 120)).loc[{'time': predators.time.min()}] = \
params['plank_pred_sizes'][:120]

# set initial detritivore spectrum (V)
detritivores.isel(size_class = slice((ind_min_detritivore_size-1), 120)).\
loc[{'time': detritivores.time.min()}] = \
np.array(params['detritivore_sizes'])[ind_min_detritivore_size-1:120]

# set initial detritus biomass density (g.m^-3) (W)
[detritus[0]] = params['init_detritus']

In [43]:
if use_init:
    # set up with the initial values from previous run
    predators.isel(size_class = slice((ind_min_pred_size-1), None)).\
    loc[{'time': predators.time.min()}] = \
    params['plank_pred_sizes'][(ind_min_pred_size-1):]
    
    # set initial detritivore spectrum from previous run
    detritivores.isel(size_class = slice((ind_min_detritivore_size-1), None)).\
    loc[{'time': detritivores.time.min()}] = \
    np.array(params['detritivore_sizes'])[(ind_min_detritivore_size-1):]

In [57]:
#intrinsic natural mortality (OM.u, OM.v)
other_mort_det = params['natural_mort']*10**(-0.25*log10_size_bins)
other_mort_pred = params['natural_mort']*10**(-0.25*log10_size_bins)

#senescence mortality rate to limit large fish from building up in the system
#same function as in Law et al 2008, with chosen parameters gives similar M2 values
#as in Hall et al. 2006 (SM.u, SM.v)
senes_mort_det = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

senes_mort_pred = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

## Fishing mortality (FVec.u, FVec.v)
From Benoit & Rochet 2004. Here `fish_mort_pred` and `fish_mort_pred` equal fixed catchability term for predators and detritivores. To be estimated along with `ind_min_det` and `ind_min_fish_pred`

In [117]:
fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': fishing_mort_pred.time.min()}] = (params['fish_mort_pred']*
                                               np.array(params['effort'][0]))

fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': fishing_mort_det.time.min()}] = (params['fish_mort_detritivore']*
                                              np.array(params['effort'][0]))

In [125]:
#output fisheries catches per yr at size - predators (Y_u)
catch_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 loc[{'time': fishing_mort_pred.time.min()}]*
 predators.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 loc[{'time': predators.time.min()}]*size_bin_vals[int(ind_min_fish_pred-1): -1])

#output fisheries catches per yr at size - detritivores (Y_v)
catch_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 loc[{'time': fishing_mort_pred.time.min()}]*
 detritivores.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 loc[{'time': predators.time.min()}]*size_bin_vals[int(ind_min_fish_det-1): -1])

In [157]:
if temp_effect:
    pel_tempeffect = np.exp(params['c1']-params['activation_energy']/
                            (params['boltzmann']*(np.array(params['sea_surf_temp'])+273)))
    ben_tempeffect = np.exp(params['c1']-params['activation_energy']/
                            (params['boltzmann']*(np.array(params['sea_floor_temp'])+273)))
else:
    pel_tempeffect = 1
    ben_tempeffect = 1

In [159]:
# ds.isel(time = 0)  lme_input$depth_m
# [ind_min_fish_det] = params['ind_min_fish_det']
# ind_min_fish_det
sum(ben_tempeffect)

416.1480922370438